In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import LSTM,Dense 
from tensorflow.keras.optimizers import Adam,Adadelta 
from sklearn.model_selection import train_test_split

In [2]:
X = np.arange(1,50) 
Y = X*15 
print("Sequence:",X,Y, sep="\n") 
X = X.reshape(-1,1,1) 
Y = Y.reshape(-1,1) 
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,shuffle=False)

Sequence:
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49]
[ 15  30  45  60  75  90 105 120 135 150 165 180 195 210 225 240 255 270
 285 300 315 330 345 360 375 390 405 420 435 450 465 480 495 510 525 540
 555 570 585 600 615 630 645 660 675 690 705 720 735]


In [3]:
def get_model(): 
    model = Sequential() 
    model.add(LSTM(50, activation='relu', input_shape=(1,1))) 
    model.add(Dense(1)) 
    model.compile(optimizer=Adadelta(learning_rate=.03), loss='mse') 
    return model 
tf.random.set_seed(0) 
model = get_model() 
print(model.summary()) 

C:\Users\gflab\New folder\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 50)                  │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

None


In [10]:
model.fit(X_train, Y_train, epochs=200, validation_split=0.2, batch_size=10) 
tf.keras.models.save_model(model,"seq_pred_model.h5") 
model  =  tf.keras.models.load_model("seq_pred_model.h5")

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0581 - val_loss: 0.5653
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0581 - val_loss: 0.5655
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0581 - val_loss: 0.5656
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0580 - val_loss: 0.5657
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0580 - val_loss: 0.5657
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0580 - val_loss: 0.5659
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0580 - val_loss: 0.5660
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0580 - val_loss: 0.5661
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0580 - val_loss: 0.5662
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0579 - val_loss: 0.5664
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0579 - val_loss: 0.5664
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0579 - val_loss

TypeError: Could not locate function 'mse'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mse', 'registered_name': 'mse'}

In [12]:
import pandas as pd 
Y_pred = model.predict(X_test, verbose=0) 
Y_diff = Y_test - Y_pred 
pd.DataFrame(np.hstack([Y_test,Y_pred,Y_diff]),columns=["Actual","Predicted","Difference"]) 

,Actual,Predicted,Difference
0,555.0,553.559937,1.440063
1,570.0,568.357483,1.642517
2,585.0,583.146484,1.853516
3,600.0,597.927734,2.072266
4,615.0,612.701660,2.298340
5,630.0,627.469177,2.530823
6,645.0,642.230713,2.769287
7,660.0,656.986694,3.013306
8,675.0,671.737671,3.262329
9,690.0,686.484131,3.515869
